In [1]:
!pip install -q transformers datasets peft bitsandbytes accelerate nltk
!pip -q install rouge-score bert-score tqdm
!pip install nltk evaluate bert_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.4 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [4]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Khoa_luan_2025_2026/Data_source/All_Datasets_utf8.csv")

#prompt instruction + latex input
def make_prompt(row):
    instr = str(row["Instruction"]).strip()
    problem = str(row["Latex Input"]).strip()

    return instr + ("\n" if not instr.endswith("\n") else "") + problem

df["prompt"] = df.apply(make_prompt, axis=1)
df["response"] = df["Response"].astype(str).str.strip()

df = df.dropna(subset=["prompt", "response"])
print("Total samples:", len(df))
print("Sample prompt:\n", df.iloc[0]["prompt"])
print("Sample response:\n", df.iloc[0]["response"])


Total samples: 785
Sample prompt:
 Hãy giúp tôi giải bài sau:
Cho hai biến cố $A$ và $B$. Biết rằng $P(A\cup B)=0.88$; $P(A)=0.6$; $P(B)=0.7$.Tính $P(A\cap B)$ và chứng tỏ $A$ và $B$ độc lập với nhau.
Sample response:
 /buoc1 Sử dụng công thức cộng xác suất: $P(AB) = P(A) + P(B) - P(A + B)$.
/buoc2 Thay số: $P(AB) = 0.6 + 0.7 - 0.88 = 0.42$.
/buoc3 Xét tích xác suất: $P(A) \cdot P(B) = 0.6 \cdot 0.7 = 0.42$.
/ketluan Vì $P(AB) = P(A) \cdot P(B) = 0.42$ nên $A$ và $B$ độc lập với nhau.
/dapan độc lập


In [5]:

train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
print("Train size:", len(train_df), "Validation size:", len(val_df))


Train size: 706 Validation size: 79


In [6]:

train_dataset = Dataset.from_pandas(train_df[["prompt", "response"]])
val_dataset   = Dataset.from_pandas(val_df[["prompt", "response"]])

#load tokenizer cho mô hình
model_name = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
# Tokenize dữ liệu
def tokenize_fn(example):
    full_text = example["prompt"] + " " + tokenizer.eos_token + " " + example["response"] + " " + tokenizer.eos_token
    #Phân tách chuỗi ký tự
    return tokenizer(full_text, truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_fn, batched=False)
val_dataset   = val_dataset.map(tokenize_fn, batched=False)

#loại bỏ cột văn bản
train_dataset = train_dataset.remove_columns(["prompt","response"])
val_dataset   = val_dataset.remove_columns(["prompt","response"])


Map:   0%|          | 0/706 [00:00<?, ? examples/s]

Map:   0%|          | 0/79 [00:00<?, ? examples/s]

In [8]:

#Cấu hình 4 bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",       #chuẩn hóa cho Qlora
    bnb_4bit_compute_dtype=torch.bfloat16,  #bfloat16 để tính toán nhằm đảm bảo tính ổn định.
    bnb_4bit_use_double_quant=False
)

#load base model 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",               #HF phân phối các lớp mô hình
    trust_remote_code=True
)

#Tắt bộ nhớ đệm và bật tính năng lưu điểm gradient để giảm mức sử dụng bộ nhớ
model.config.use_cache = False  #ko sử dụng bộ nhớ đệm, tiết kiệm bộ nhớ
model.gradient_checkpointing_enable()  # trade compute for memory by checkpointing layers


config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:

#training 4-bit
model = prepare_model_for_kbit_training(model)

#cấu hình lora
lora_config = LoraConfig(
    r=64,
    lora_alpha=16,    #hệ số alpha mặc định 16
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"],  #lớp lora
    lora_dropout=0.1, #giảm trọng lượng lora
    bias="none",
    task_type="CAUSAL_LM")
#bọc mô hình bằng bộ chuyển đổi peft lora
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 92,274,688 || all params: 7,340,298,240 || trainable%: 1.2571


In [10]:

batch_size = 4
gradient_accumulation = 4
epochs = 3.12
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_out",
    overwrite_output_dir=True,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation,  #hàm học tích lũy
    learning_rate=2e-4,  #LR trong cho LoRA
    bf16=True,
    optim="paged_adamw_32bit",  #tối ưu hóa thuật toán Adam
    logging_steps=10,
    save_steps=15,
    eval_strategy="steps",
    save_strategy="steps"
)

#DataCollator cho mô hình ngôn ngữ để xử lý khoảng trắng và giấu nhãn
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer
)

/tmp/ipython-input-813869951.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [12]:
trainer.train(resume_from_checkpoint="/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_out/checkpoint-141")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.
	eval_steps: 10 (from args) != 500 (from trainer_state.json)
	save_steps: 15 (from args) != 10 (from trainer_state.json)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 3


wandb: You chose "Don't visualize my results"


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss,Validation Loss


TrainOutput(global_step=142, training_loss=0.000950890107893608, metrics={'train_runtime': 284.9202, 'train_samples_per_second': 7.731, 'train_steps_per_second': 0.495, 'total_flos': 4.852672619475763e+16, 'train_loss': 0.000950890107893608, 'epoch': 3.15819209039548})

In [ ]:
#lưu model

#trainer.save_model("/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final")

#tokenizer.save_pretrained("/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final")


('/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final/tokenizer_config.json',
 '/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final/special_tokens_map.json',
 '/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final/chat_template.jinja',
 '/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final/tokenizer.model',
 '/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final/added_tokens.json',
 '/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final/tokenizer.json')

In [ ]:
#trainer.train()


In [ ]:
#tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final")

#model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Khoa_luan_2025_2026/mistral_final")

config.json:   0%|          | 0.00/601 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
import random
import torch
from nltk.translate.bleu_score import corpus_bleu


generated_texts = []
reference_texts = []

model.eval()
for sample in val_df.sample(1).itertuples(index=False): #lấy 1 mẫu ngẫu nhiên
    prompt = sample.prompt
    ref = sample.response
    #tokenize promt đầu vào
    inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=500,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
        )

    #tạo câu trả lời do máy học được (gen_text)
    gen_ids = out[0, inputs["input_ids"].shape[1]:]
    gen_text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()

    generated_texts.append(gen_text)
    reference_texts.append([ref])

    print(f"Prompt: {prompt}\nGenerated: {gen_text}\nReference: {ref}\n{'-'*60}")

# tính BLEU
gen_tokens = [g.split() for g in generated_texts]
ref_tokens = [[r.split() for r in refs] for refs in reference_texts]
bleu = corpus_bleu(ref_tokens, gen_tokens)
print(f"BLEU score: {bleu:.4f}")


Prompt: Hãy giải giúp tôi bài sau:
Theo dõi thời gian hoàn thành sản phẩm của $36$ công nhân tại phân xưởng X: thời gian gia công trung bình $13.2$ phút/sản phẩm; độ lệch chuẩn hiệu chỉnh $1.25$ phút. Định mức phân xưởng đề ra $14$ phút/sản phẩm; với mức ý nghĩa $5\%$ có phải thay đổi định mức đó không?
Generated: Đáp: Không. Để giải bài này, ta có các giả thiết:

1. $H_0: a = 14$; $W_\alpha = (-\infty; -1.645); H_1: a \neq 14$.
2. $U = \frac{(\bar{x} - a_0)\sqrt{n}}{S'} \sim N(0, 1)$.
3. $U_{qs} = -1.25$ không thuộc $W_\alpha \rightarrow$ chấp nhận $H_0$.

Kết luận: Không thay đổi định mức.

\begin{table}[!htbp]
\centering
\renewcommand{\arraystretch}{1.2}
\setlength{\tabcolsep}{12pt}
\begin{tabular}{|c|c|c|c|c|c|c|c|}
\hline
$X_i$ & $1$ & $2$ & $3$ & $4$ & $5$ & $6$ & $7$ \\ \hline
$Y_i$ & $1$ & $1$ & $2$ & $2$ & $3$ & $3$ & $4$ \\ \hline
\end{tabular}
\end{table}

\begin{table}[!htbp]
\centering
\renewcommand{\arraystretch}{1.2}
\setlength{\tabcolsep}{12pt}
\begin{tabular}{|c|c|c|c|

In [15]:
import nltk
from nltk.translate.bleu_score import corpus_bleu


generated_texts2 = []
reference_texts2 = []
for sample2 in val_df.sample(1).itertuples():  #lấy 1 mẫu ngẫu nhiền
    prompt2 = sample2.prompt
    # Tokenize prompt
    inputs2 = tokenizer(prompt2, return_tensors="pt").to(model.device)
    outputs2 = model.generate(**inputs2, max_new_tokens=512, do_sample=False)
    gen_text2 = tokenizer.decode(outputs2[0], skip_special_tokens=True)
    # The generated text includes the prompt + response (because we trained the model in a prompt->response fashion).
    # We extract the part after the prompt to get the answer.
    generated_answer2 = gen_text2[len(prompt2):].strip()
    generated_texts2.append(generated_answer2)
    reference_texts2.append([sample2.response])  # reference as a one-item list for BLEU

    print(f"Prompt: {prompt2}\nGenerated answer: {generated_answer2}\nReference answer: {sample2.response}\n{'-'*60}")

# Compute corpus BLEU score for all generated vs reference texts
# Tokenize by whitespace for BLEU calculation:
gen_tokens2 = [gen.split() for gen in generated_texts2]
ref_tokens2 = [[ref.split() for ref in refs] for refs in reference_texts2]
bleu_score = corpus_bleu(ref_tokens2,gen_tokens2)
print(f"BLEU score: {bleu_score:.4f}")

Prompt: Hãy giúp tôi giải bài sau:
Cho hệ đầy đủ ba biến cố $\{A, B, C\}$ với $P(A) = 0.3$; $P(B) = 2P(A)$. Biết biến cố $F$ thỏa mãn: $P(F|A) = 0.02$; $P(F|B) = 0.35$; $P(F|C) = 0.5$. Biết $P(F) = 0.266$. Tính xác suất $P(\bar{A}|\bar{F})$?
Generated answer: Đáp: $0.966$. Để giải bài này, ta có các tính chất hệ đầy đủ:

1. Tổng xác suất của hệ đầy đủ: $P(A) + P(B) + P(C) = 1$.
2. Các biến cố thỏa mãn: $P(A) \cdot P(\bar{A}|B) = P(A) \cdot P(\bar{A}|C) = P(B) \cdot P(\bar{B}|A) = P(C) \cdot P(\bar{C}|B)$.
3. Các xác suất có điều kiện đối thiết: $P(\bar{A}|B) = \frac{P(B) - P(A)}{P(B)} = \frac{2P(A) - P(A)}{2P(A)} = \frac{P(A)}{2P(A)} = 0.5$.
4. Tính được $P(\bar{A}|C) = 0.5$.
5. Áp dụng công thức Bayes: $P(A|\bar{F}) = \frac{P(A) \cdot P(\bar{F}|A)}{P(\bar{F})} = \frac{0.3 \cdot 0.02}{0.266} = 0.06$.
6. Sử dụng tính chất đối thiết: $P(\bar{A}|\bar{F}) = 1 - P(A|\bar{F}) = 1 - 0.06 = 0.94$.
7. Dùng tính chất 1: $P(\bar{F}) = 1 - P(F) = 1 - 0.266 = 0.734$.
8. Thay số vào công thức: $P(\b

In [23]:
# Hàm sinh câu trả lời từ prompt mới
generator1 = pipeline("text-generation", model=model, tokenizer=tokenizer)

def generate_answer1(prompt_text):
    result = generator1(prompt_text,max_new_tokens=512, max_length=512, num_return_sequences=1)
    answer = result[0]['generated_text']
    answer = answer.replace(prompt_text, "").strip()
    return answer
instruction="Hãy giúp tôi giải bải sau:"
new_question = "Cho hàm mật độ của biến ngẫu nhiên $X$:$f(x)=\begin{cases}kx(4-x)&x\in[0,4]\\0&x\notin[0,4]\end{cases}$ với $k=\dfrac{3}{32}$. Tính $E(X)$ và $P(-2<X<3)$."
new_question = instruction + "\n" + new_question
new_answer = generate_answer1(new_question)
print("Question:", new_question)
print("Response:", new_answer)


<>:10: SyntaxWarning: invalid escape sequence '\i'
<>:10: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipython-input-3032312270.py:10: SyntaxWarning: invalid escape sequence '\i'
  new_question = "Cho hàm mật độ của biến ngẫu nhiên $X$:$f(x)=\begin{cases}kx(4-x)&x\in[0,4]\\0&x\notin[0,4]\end{cases}$ với $k=\dfrac{3}{32}$. Tính $E(X)$ và $P(-2<X<3)$."
Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Question: Hãy giúp tôi giải bải sau:
Cho hàm mật độ của biến ngẫu nhiên $X$:$f(x)=egin{cases}kx(4-x)&x\in[0,4]\0&x
otin[0,4]\end{cases}$ với $k=\dfrac{3}{32}$. Tính $E(X)$ và $P(-2<X<3)$.
Response: Đáp
$E(X)=\frac{16}{15}$, $P(-2<X<3)=\frac{19}{32}$.

\begin{boxed}\begin{itemize}
\item Biến cố $X\in[0,4]$ xác định trên tập $x\in[0,4]$.
\item $f(x)=\frac{3}{32}x(4-x)$ nếu $x\in[0,4]$.
\item $k=\frac{3}{32}$.
\item Tính $E(X)=\int xf(x)dx=\frac{16}{15}$.
\item $P(-2<X<3)=\int_{0}^{4}f(x)dx=\frac{19}{32}$.
\end{itemize}\end{boxed}

\textbf{Thảo luận:}
- Biến cố $X\in[0,4]$ xác định trên tập $x\in[0,4]$.
- H


In [18]:

import nltk
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from evaluate import load
from bert_score import BERTScorer


nltk.download('wordnet')
nltk.download('punkt')

# 5. Sinh đáp án với mô hình và lưu kết quả
predictions = []
references = []
for item in df.itertuples(index=False):
    prompt = item.prompt
    true_answer = item.response
    # Sinh câu trả lời từ mô hình (có thể điều chỉnh max_length, do_sample, v.v.)
    result = generator(prompt, max_length=50, num_return_sequences=1)
    pred = result[0]['generated_text']
    # Xử lý để loại bỏ prompt khỏi output nếu cần
    pred = pred.replace(prompt, "").strip()
    predictions.append(pred)
    references.append(true_answer)

# 6. Tính các chỉ số đánh giá
# 6.1 BLEU (dùng Evaluate, đầu vào là token lists):contentReference[oaicite:2]{index=2}
bleu = load("bleu")
# Chuyển chuỗi thành danh sách token; Evaluate BLEU cần dạng [[token,...],...]
bleu_preds = [p.split() for p in predictions]
bleu_refs = [[ref.split()] for ref in references]  # mỗi ref cũng là danh sách token trong một list
bleu_result = bleu.compute(predictions=bleu_preds, references=bleu_refs)
print("BLEU score:", bleu_result["bleu"])

# 6.2 ROUGE (Evaluate nhận input dạng chuỗi):contentReference[oaicite:3]{index=3}
rouge = load("rouge")
rouge_result = rouge.compute(predictions=predictions, references=references)
print("ROUGE scores:", rouge_result)

# 6.3 METEOR (Evaluate, hoặc NLTK):contentReference[oaicite:4]{index=4}
meteor = load("meteor")
meteor_result = meteor.compute(predictions=predictions, references=references)
print("METEOR score:", meteor_result["meteor"])

# 6.4 BERTScore (Evaluate, cần chỉ định ngôn ngữ, ví dụ "en" cho tiếng Anh):contentReference[oaicite:5]{index=5}
bertscore = load("bertscore")
bertscore_result = bertscore.compute(predictions=predictions, references=references, lang="en")
print("BERTScore F1:", [round(score, 4) for score in bertscore_result["f1"]])


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_to

KeyboardInterrupt: 

In [24]:
import torch
import evaluate
import pandas as pd
import gc
from tqdm import tqdm

full_dataset = Dataset.from_pandas(df)
dataset = full_dataset.train_test_split(test_size=0.01)["train"]
try:
    test_data = dataset.select(range(len(dataset)-5, len(dataset)))
except:
    test_data = dataset.select(range(min(5, len(dataset))))

print(f" Đang đánh giá trên {len(test_data)} mẫu dữ liệu...")

predictions = []
references = []

print(" Đang sinh câu trả lời từ model...")
for item in tqdm(test_data):
    # Lấy Input
    key_map = {k.lower().strip(): k for k in item.keys()}
    inst = str(item[key_map.get('instruction', 'instruction')])
    inp  = str(item[key_map.get('latex input', 'latex input')])

    #đáp án gốc referece
    ground_truth = str(item[key_map.get('response', 'response')])
    references.append(ground_truth)

    # Tạo Prompt
    user_content = f"{inst}\n\n{inp}".strip()
    messages = [{"role": "user", "content": user_content}]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # Generate
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            temperature=0.1,
            top_p=0.9
        )

    # Decode
    generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    predictions.append(response)

#GIẢI PHÓNG VRAM
del model
del tokenizer
gc.collect()
torch.cuda.empty_cache()

#tÍNH TOÁN CÁC METRICS
# Load metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
meteor = evaluate.load("meteor")
bertscore = evaluate.load("bertscore")

# Tính BLEU
results_bleu = bleu.compute(predictions=predictions, references=references)

# Tính ROUGE
results_rouge = rouge.compute(predictions=predictions, references=references)

# Tính METEOR
results_meteor = meteor.compute(predictions=predictions, references=references)

# Tính BERTScore (Dùng model đa ngôn ngữ)
# lang="vi" sẽ tự động tải model bert-base-multilingual-cased
results_bert = bertscore.compute(predictions=predictions, references=references, lang="vi")

#HIỂN THỊ KẾT QUẢ
print("\n" + "="*40)
print(" KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH (FINAL REPORT)")
print("="*40)

print(f"BLEU Score:  {results_bleu['bleu']:.4f}")
print(f"ROUGE-1:     {results_rouge['rouge1']:.4f} (Độ trùng khớp từ đơn)")
print(f"ROUGE-L:     {results_rouge['rougeL']:.4f} (Độ trùng khớp cấu trúc câu)")
print(f"METEOR:      {results_meteor['meteor']:.4f} (Độ tương đồng ngữ nghĩa cơ bản)")
print(f"BERTScore F1:{sum(results_bert['f1']) / len(results_bert['f1']):.4f} (Độ tương đồng ngữ nghĩa sâu)")


 Đang đánh giá trên 5 mẫu dữ liệu...
 Đang sinh câu trả lời từ model...


100%|██████████| 5/5 [07:12<00:00, 86.55s/it]


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


 KẾT QUẢ ĐÁNH GIÁ MÔ HÌNH (FINAL REPORT)
BLEU Score:  0.2961
ROUGE-1:     0.5130 (Độ trùng khớp từ đơn)
ROUGE-L:     0.3480 (Độ trùng khớp cấu trúc câu)
METEOR:      0.4049 (Độ tương đồng ngữ nghĩa cơ bản)
BERTScore F1:0.7613 (Độ tương đồng ngữ nghĩa sâu)
